In [3]:
import serial
import csv
import time
import datetime
from jupyterplot import ProgressPlot

In [4]:
arduino_port = "/dev/ttyUSB1"
arduino_baudrate = 115200
arduino_serial = serial.Serial(arduino_port, arduino_baudrate)
data_header = ["LL", "LR", "DT", "AX", "AZ"]
file_path = ""

pp = ProgressPlot(plot_names=["Light Sensor", "LiDAR Distance"],
                  line_names=["L", "R"])

def show_data_with_csv():
    try:
        with open(file_path + datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S") + ".csv",mode="w",newline="") as file:
            csv_writer = csv.writer(file)
            csv_writer.writerow(["Time"]  + data_header)
            start_time = time.time()
            while time.time() - start_time < 120:
                data = arduino_serial.readline().decode("utf-8").strip()
                if data.count(",") == len(data_header) - 1:
                    try:
                        LL, LR, DT, AX, AZ = map(lambda x: float(x[3:]),data.split(","),)
                        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                        csv_writer.writerow([timestamp, LL, LR, DT, AX, AZ])
                        pp.update([[LL, LR],[DT, DT]])
                    except ValueError as e:
                        print(f"Invalid data format: {data}; {e}")
                else:
                    print(f"Invalid data count: {data}")
                time.sleep(0.01)
            pp.finalize()
    except KeyboardInterrupt:
        arduino_serial.close()

def show_data():
    try:
        while True:
            data = arduino_serial.readline().decode("utf-8").strip()
            if data.count(",") == len(data_header) - 1:
                try:
                    LL, LR, DT, AX, AZ = map(lambda x: float(x[3:]),data.split(","),)
                    pp.update([[LL, LR],[DT, DT]])
                except ValueError as e:
                    print(f"Invalid data format: {data}; {e}")
            else:
                print(f"Invalid data count: {data}")
            time.sleep(0.001)
            pp.finalize()
    except KeyboardInterrupt:
        arduino_serial.close()

show_data()

IndentationError: unexpected indent (2480454995.py, line 45)